# Data Mining Challenge

* Student Name: Michael Rideout
* Student Number: 225065259
* E-mail: s225065259@deakin.edu.au
* Student Course Code: SIT731
---

# Introduction

This investigation will explore interesting insights gleamed through merging and analysing multiple NHANES [1] datasets. These insights will be visualised using the Python Bokeh [2] package.
The NHANES or Ntional Health and Nutrition Examination Survey, is a comprehensive sample from the U.S. population of health, nutritional and demographic data. The datasets used in this investigation were for the period of August 2021 to August 2023. The Bokeh package is a powerful visualisation library that allows users to build dashboards, plots and other visualisations. This investigation will focus primarily on exploring if socio-economic groups have differential life style choices or health outcomes. 



# Data

The NAHNES datasets that were chosen for this investigation were:
|Dataset Name|Dataset Documentation URL|
|------------|-----------------|
|Demographics|https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DEMO_L.htm|
|Body Measures|https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BMX_L.htm|
|Alcohol Use|https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/ALQ_L.htm|
|Medical Conditions|https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/MCQ_L.htm|
|Smoking - Cigarette Use|https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/SMQ_L.htm|


In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show 
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Select, Legend, Span, Whisker, HoverTool, CustomJSTickFormatter, Label
from bokeh.io import output_notebook, show
from bokeh.palettes import Category10
from bokeh.transform import factor_cmap
from bokeh.palettes import DarkText, TolRainbow7 as colors


In [2]:
# Define some constant strings
SOCIOECONOMIC_GROUP = "socioeconomic_group"
SEQN = "seqn"

In [3]:
demographics_url = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DEMO_L.xpt'
body_measures_url = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BMX_L.xpt'
alcohol_use_url = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/ALQ_L.xpt'
medical_conditions_url = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/MCQ_L.xpt'
smoking_url = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/SMQ_L.xpt'

# Dataset Preprocessing

## Demographics Dataset Processing

The Demographic dataset provides various demographic information about a sample from the U.S. population.

In [4]:
# process demographics dataset
demo_dataset_df = pd.read_sas(demographics_url, format='xport')

print(f"Shape: {demo_dataset_df.shape}")
demo_dataset_df.info()


Shape: (11933, 27)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11933 entries, 0 to 11932
Data columns (total 27 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      11933 non-null  float64
 1   SDDSRVYR  11933 non-null  float64
 2   RIDSTATR  11933 non-null  float64
 3   RIAGENDR  11933 non-null  float64
 4   RIDAGEYR  11933 non-null  float64
 5   RIDAGEMN  377 non-null    float64
 6   RIDRETH1  11933 non-null  float64
 7   RIDRETH3  11933 non-null  float64
 8   RIDEXMON  8860 non-null   float64
 9   RIDEXAGM  2787 non-null   float64
 10  DMQMILIZ  8301 non-null   float64
 11  DMDBORN4  11914 non-null  float64
 12  DMDYRUSR  1875 non-null   float64
 13  DMDEDUC2  7794 non-null   float64
 14  DMDMARTZ  7792 non-null   float64
 15  RIDEXPRG  1503 non-null   float64
 16  DMDHHSIZ  11933 non-null  float64
 17  DMDHRGND  4115 non-null   float64
 18  DMDHRAGZ  4124 non-null   float64
 19  DMDHREDZ  3746 non-null   float64
 20  DMDHRMAZ 

Appendix 1 lists all the features in the demographic dataset. The features that we are interested in are:
- "SEQN" - unique person idenifier
- "RIAGENDR" - Gender
- "RIDAGEYR" - Age in years
- "DMDEDUC2" - Education level (for 20 year olds or over)
- "INDFMPIR"  - Family income to poverty ratio

Socio-economic group is computed from the family income to poverty ratio feature. This ratio is grouped by 33% quantiles into 3 socio-economic groups 'lower', 'middle', 'upper'

In [5]:
# We are only interested in the following fields
# "SEQN" - unique person idenifier
# "RIAGENDR" - Gender
# "RIDAGEYR" - Age in years
# "DMDEDUC2" - Education level (for 20 year olds or over)
# "INDFMPIR"  - Family income to poverty ratio
demo_df = demo_dataset_df[["SEQN", "RIAGENDR", "RIDAGEYR", "DMDEDUC2", "INDFMPIR"]].copy()
# Change the column names to something more readable
demo_df.columns = ["seqn", "gender", "age", "education", "poverty_ratio"]

# Set the correct datatypes instead of the default float type
demo_df["seqn"] = demo_df["seqn"].astype(int)
demo_df["age"] = demo_df["age"].astype(int)

# Replace gender nomials values with strings
demo_df["gender"].replace({1: "Male", 2: "Female"}, inplace=True)

# Create new socio economic group feature based on poverty ratio tertiles (33% quantiles)
demo_df['socioeconomic_group'] = pd.qcut(demo_df['poverty_ratio'], 
                                        q=3, 
                                        labels=['lower', 'middle', 'upper'])

demo_df.head()

/tmp/ipykernel_51936/1644238866.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  demo_df["gender"].replace({1: "Male", 2: "Female"}, inplace=True)


,seqn,gender,age,education,poverty_ratio,socioeconomic_group
0,130378,Male,43,5.0,5.00,upper
1,130379,Male,66,5.0,5.00,upper
2,130380,Female,44,3.0,1.41,lower
3,130381,Female,5,NaN,1.53,lower
4,130382,Male,2,NaN,3.60,middle


## Body Measures Dataset Preprocessing

The Body Measures dataset is a result of physical examination of the survey participants.

In [6]:
# process body measures dataset
body_measures_dataset_df = pd.read_sas(body_measures_url, format='xport')

print(f"Shape: {body_measures_dataset_df.shape}")
body_measures_dataset_df.info()

Shape: (8860, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8860 entries, 0 to 8859
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      8860 non-null   float64
 1   BMDSTATS  8860 non-null   float64
 2   BMXWT     8754 non-null   float64
 3   BMIWT     345 non-null    float64
 4   BMXRECUM  454 non-null    float64
 5   BMIRECUM  18 non-null     float64
 6   BMXHEAD   70 non-null     float64
 7   BMIHEAD   0 non-null      float64
 8   BMXHT     8499 non-null   float64
 9   BMIHT     134 non-null    float64
 10  BMXBMI    8471 non-null   float64
 11  BMDBMIC   2492 non-null   float64
 12  BMXLEG    7335 non-null   float64
 13  BMILEG    396 non-null    float64
 14  BMXARML   8568 non-null   float64
 15  BMIARML   200 non-null    float64
 16  BMXARMC   8562 non-null   float64
 17  BMIARMC   205 non-null    float64
 18  BMXWAIST  8190 non-null   float64
 19  BMIWAIST  347 non-null    float64
 20  BMXHIP    67

Appendix 2 lists all the features in the body measures dataset. The features that we are interested in are:
- "SEQN" - unique person identifier
- "BMXBMI" - BMI measure of the person

From BMXBMI a new feature is computed, bmi_category. It groups BMI measures into one of the following categories that provide an indication of weight health, 'underweight', 'normal', 'overweight', 'obese'.

In [7]:
# We are only interested in the following fields
# "SEQN" - unique person idenifier
# "BMXBMI" - Body Mass Index (kg/m^2)high
bmi_df = body_measures_dataset_df[["SEQN", "BMXBMI"]]

# Rename columns
bmi_df.columns = ["seqn", "bmi"]

# Create BMI categories based on WHO classification
# Underweight = <18.5
# Normal weight = 18.5-24.9 
# Overweight = 25-29.9
# Obese = BMI of 30 or greater
bmi_df['bmi_category'] = pd.cut(bmi_df['bmi'],
                               bins=[-float('inf'), 18.5, 24.9, 29.9, float('inf')],
                               labels=['underweight', 'normal', 'overweight', 'obese'])
bmi_df.head()

/tmp/ipykernel_51936/2230951975.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bmi_df['bmi_category'] = pd.cut(bmi_df['bmi'],


,seqn,bmi,bmi_category
0,130378.0,27.0,overweight
1,130379.0,33.5,obese
2,130380.0,29.7,overweight
3,130381.0,23.8,normal
4,130382.0,NaN,NaN


## Alcohol Use Dataset Preprocessing

The Alochol Use dataset provides participants responses to questionnaire questions regarding lifetime and current alcohol use.

In [8]:
# process alcohol use dataset
alcohol_use_dataset_df = pd.read_sas(alcohol_use_url, format='xport')

print(f"Shape: {alcohol_use_dataset_df.shape}")
alcohol_use_dataset_df.info()

Shape: (6337, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6337 entries, 0 to 6336
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SEQN    6337 non-null   float64
 1   ALQ111  5481 non-null   float64
 2   ALQ121  4922 non-null   float64
 3   ALQ130  4069 non-null   float64
 4   ALQ142  4082 non-null   float64
 5   ALQ270  2366 non-null   float64
 6   ALQ280  2362 non-null   float64
 7   ALQ151  4901 non-null   float64
 8   ALQ170  2358 non-null   float64
dtypes: float64(9)
memory usage: 445.7 KB


Appendix 3 lists all the features in the alcohol use dataset. The features that we are interested in are:
- "SEQN" - Unique person identifier
- "ALQ121" - Past 12 months how often drink alcohol beverage
- "ALQ130" - Average number of alcoholic drinks per period in the last 12 months

A new feature is computed, drink_amount. This is an estimate of the annual total alcohol consumption in standard units of alcohol. It is derived from both the drink frequency (ALQ121) and drink amount (ALQ130) features.

In [9]:
# Drop all but these fields SEQN, ALQ121, ALQ130
alcohol_use_df = alcohol_use_dataset_df[["SEQN", "ALQ121", "ALQ130"]]

# Change the column names to something more readable
alcohol_use_df.columns = ["seqn", "drink_frequency", "drink_amount"]

# Drop na records
alcohol_use_df.dropna(subset=["seqn", "drink_frequency", "drink_amount"], inplace=True)

# Remove records where drink_frequency is 99 (refused to answer)
alcohol_use_df = alcohol_use_df[alcohol_use_df['drink_frequency'] != 99]

# Remove records where drink_amount is 77 (refused) or 99 (don't know)
alcohol_use_df = alcohol_use_df[~alcohol_use_df['drink_amount'].isin([77, 99])]



# Set the correct datatypes instead of the default float type
alcohol_use_df["seqn"] = alcohol_use_df["seqn"].astype(int)
alcohol_use_df["drink_frequency"] = alcohol_use_df["drink_frequency"].astype(int)
alcohol_use_df['drink_amount'] = alcohol_use_df["drink_amount"].astype(int)



# Create a frequency mapping dictionary that converts drink frequency codes to annual occurrences
frequency_map = {
    0: 0,      # never
    1: 365,    # Every day
    2: 312,    # Nearly every day (~6 times/week)
    3: 182,    # 3-4 times a week (average 3.5)
    4: 104,    # 2 times a week
    5: 52,     # Once a week
    6: 30,     # 2-3 times a month (average 2.5)
    7: 12,     # Once a month
    8: 9,      # 7-11 times/year (average 9)
    9: 4.5,    # 3-6 times/year (average 4.5)
    10: 1.5,    # 1-2 times a year (average 1.5)
}

# Calculate annual consumption by multiplying frequency by amount
# Filter out non-valid frequency codes (0, 77, 99) first
valid_freq_mask = alcohol_use_df['drink_frequency'].isin(frequency_map.keys())
alcohol_use_df = alcohol_use_df[valid_freq_mask]

alcohol_use_df['annual_use'] = alcohol_use_df.apply(
    lambda row: frequency_map[row['drink_frequency']] * row['drink_amount'],
    axis=1
)

alcohol_use_df.head()

/tmp/ipykernel_51936/343098040.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alcohol_use_df.dropna(subset=["seqn", "drink_frequency", "drink_amount"], inplace=True)


,seqn,drink_frequency,drink_amount,annual_use
1,130379,2,3,936.0
2,130380,10,1,1.5
3,130386,4,2,208.0
6,130389,2,2,624.0
9,130392,8,1,9.0


## Medical Conditions Dataset Preprocessing

The Medical Conditions dataset is produced by participants interviews querying their health and medical history.

In [10]:
# process hospital access dataset
medical_conditions_dataset_df = pd.read_sas(medical_conditions_url, format='xport')

print(f"Shape: {medical_conditions_dataset_df.shape}")
medical_conditions_dataset_df.info()

Shape: (11744, 35)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11744 entries, 0 to 11743
Data columns (total 35 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   SEQN     11744 non-null  float64
 1   MCQ010   11743 non-null  float64
 2   MCQ035   1946 non-null   float64
 3   MCQ040   1219 non-null   float64
 4   MCQ050   1219 non-null   float64
 5   AGQ030   11743 non-null  float64
 6   MCQ053   11741 non-null  float64
 7   MCQ149   434 non-null    float64
 8   MCQ160A  7807 non-null   float64
 9   MCQ195   2532 non-null   float64
 10  MCQ160B  7808 non-null   float64
 11  MCQ160C  7807 non-null   float64
 12  MCQ160D  7808 non-null   float64
 13  MCQ160E  7807 non-null   float64
 14  MCQ160F  7806 non-null   float64
 15  MCQ160M  7806 non-null   float64
 16  MCQ170M  1053 non-null   float64
 17  MCQ160P  7807 non-null   float64
 18  MCQ160L  7807 non-null   float64
 19  MCQ170L  425 non-null    float64
 20  MCQ500   1578 non-null   float6

Appendix 4 lists all the features in the medical conditions dataset. The features used in this investigation were:
- "SEQN" - Unique person identifier
- "MCQ160B" - Congestive heart failure
- "MCQ160C" - Coronary heart disease
- "MCQ160D" - Agina pectoris
- "MCQ160E" - Heart attack
- "MCQ220" - Cancer

A new feature, heart_condition, was computed as being true when the respondent answer true for having any heart condition.

In [11]:
# Drop all but these fields SEQN, MCQ160b, MCQ160c,MCQ160d, MCQ160e (heart diseases), MCQ220
medical_conditions_df = medical_conditions_dataset_df[["SEQN", "MCQ160B", "MCQ160C", "MCQ160D", "MCQ160E", "MCQ220"]]

# Rename columns
medical_conditions_df.columns = ["seqn", "heart_congestive", "heart_coronary", "heart_angina", "heart_attack", "cancer"]

# Create heart_condition column that is 1 if any heart condition is present (1), 0 otherwise
medical_conditions_df['heart_condition'] = ((medical_conditions_df['heart_congestive'] == 1) |
                                          (medical_conditions_df['heart_coronary'] == 1) |
                                          (medical_conditions_df['heart_angina'] == 1) |
                                          (medical_conditions_df['heart_attack'] == 1)).astype(int)


medical_conditions_df.head()

/tmp/ipykernel_51936/341397135.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medical_conditions_df['heart_condition'] = ((medical_conditions_df['heart_congestive'] == 1) |


,seqn,heart_congestive,heart_coronary,heart_angina,heart_attack,cancer,heart_condition
0,130378.0,2.0,2.0,2.0,2.0,2.0,0
1,130379.0,2.0,2.0,2.0,2.0,1.0,0
2,130380.0,2.0,2.0,2.0,2.0,2.0,0
3,130381.0,NaN,NaN,NaN,NaN,NaN,0
4,130382.0,NaN,NaN,NaN,NaN,NaN,0


## Smoking Dataset Preprocessing

The smoking - cigarette use dataset was produced by asking participants of the survey about their cigarette smoking habbits and history

In [12]:
# process smoking dataset
smoking_dataset_df = pd.read_sas(smoking_url, format='xport')

print(f"Shape: {smoking_dataset_df.shape}")
smoking_dataset_df.info()

Shape: (9015, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9015 entries, 0 to 9014
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      9015 non-null   float64
 1   SMQ020    8135 non-null   float64
 2   SMQ040    3243 non-null   float64
 3   SMD641    273 non-null    float64
 4   SMD650    1185 non-null   float64
 5   SMD100MN  1175 non-null   float64
 6   SMQ621    772 non-null    float64
 7   SMD630    23 non-null     float64
 8   SMAQUEX2  9015 non-null   float64
dtypes: float64(9)
memory usage: 634.0 KB


Appendix 5 lists all the features of the smoking - cigarette use dataset. The features used in this analysis were:
- "SEQN" - Unique person identifier
- "SMQ020" - Smoked 100 cigarettes in life
- "SMQ040" - Smoke frequency
- "SMD641" - Smoke amount

A new feature, smoke_use, was computed as an estimate for the total number of cigarettes a person would smoke in a year.

In [13]:
# Keep only the following fields SEQN, SMQ040, SMD641
smoking_df = smoking_dataset_df[["SEQN", "SMQ020", "SMQ040", "SMD641"]]

# Rename columns
smoking_df.columns = ["seqn", "smoked_in_life", "smoke_frequency", "smoke_amount"]

# Drop rows with missing values or invalid codes
smoking_df = smoking_df[
    (smoking_df['smoked_in_life'].isin([1, 2, 3]))
]

# Calculate annual cigarette use:
# For daily smokers (1): smoke_amount * 365
# For some days smokers (2): smoke_amount * 52 (assuming once per week)
# For non-smokers (3): 0
smoking_df['smoke_use'] = np.where(
    smoking_df['smoke_frequency'] == 1, smoking_df['smoke_amount'] * 365,
    np.where(smoking_df['smoke_frequency'] == 2, smoking_df['smoke_amount'] * 52, 0)
)


## Data Analysis

### BMI Analysis

Body Mass Index (BMI) is a standarised valued dervide from the weight and height of a person. Major adult BMI classifications are underweight (under 18.5 kg/m2), normal weight (18.5 to 24.9), overweight (25 to 29.9), and obese (30 or more) [3]. An interactive bokeh graph of BMI vs Education Level by Socio-economic Group was constructed to determine if there is any discernable relationship between BMI, education level and socio-economic groups.

In [14]:
# Merge demo_df and bmi_df
merged_bmi_df = pd.merge(demo_df, bmi_df, on='seqn')

output_notebook()

source = ColumnDataSource(merged_bmi_df)

p = figure(title='BMI vs Education Level by Socio-economic Group',
           x_axis_label='Education Level',
           y_axis_label='BMI',
           width=800, height=600)

colours = Category10[3] 
socioeconomic_groups = ["lower", "middle", "upper"]

# Create scatter plots for each socioeconomic group
scatter_renderers = []
for group, colour in zip(socioeconomic_groups, colours):
    mask = merged_bmi_df['socioeconomic_group'] == group
    scatter = p.scatter('education', 'bmi',
                       source=ColumnDataSource(merged_bmi_df[mask]),
                       color=colour,
                       legend_label=group,
                       size=8,
                       alpha=0.6)
    scatter_renderers.append(scatter)

p.legend.click_policy = "hide"  # Enable hiding groups by clicking legend
p.legend.location = "top_right"

show(p)



Loading BokehJS ...

As the bokeh scatter plot is interactive, socio-economic groups could be enabled or disabled via legend toggles. From utilising this toggle, it can be observed that:
- There is no discernable difference in the spread of BMI when comparing socio-economic groups
- There is a discernable difference between socio-economic groups and the level of education achieved. Lower vs upper socio-economic groups show that those in the lower group attained lower levels of education than those in the upper group.

A further drill down into socio-economic group and BMI category is prudent. A bar chart of the percentage people in each of the bmi categories per socio-economic group may be enlightening.

In [15]:
# Calculate percentages for each socioeconomic group and BMI category

grouped = merged_bmi_df.groupby(['socioeconomic_group', 'bmi_category']).size().reset_index(name='count')
total_by_group = merged_bmi_df.groupby('socioeconomic_group').size()
grouped['percentage'] = grouped.apply(lambda x: x['count'] / total_by_group[x['socioeconomic_group']] * 100, axis=1)


p = figure(title='BMI Category Distribution by Socio-economic Group',
           x_range=[],  # Will be set after data processing
           y_axis_label='Percentage (%)',
           width=800, height=600)

# Create x-range categories
x_categories = []
for group in grouped['socioeconomic_group'].unique():
    for category in grouped[grouped['socioeconomic_group'] == group]['bmi_category'].unique():
        x_categories.append(f"{group} - {category}")
p.x_range.factors = x_categories

colours = Category10[3]
colour_map = dict(zip(['lower', 'middle', 'upper'], colours))

# Create the bars with different colors per socioeconomic group
p.vbar(x=[f"{row['socioeconomic_group']} - {row['bmi_category']}" for _, row in grouped.iterrows()],
       top=grouped['percentage'],
       width=0.8,
       fill_color=[colour_map[row['socioeconomic_group']] for _, row in grouped.iterrows()],
       line_color='black')


p.xgrid.grid_line_color = None
p.xaxis.axis_label = 'Socioeconomic Group - BMI Category'
p.xaxis.major_label_orientation = 45
p.title.text_font_size = '14pt'

show(p)



/tmp/ipykernel_51936/1212948923.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = merged_bmi_df.groupby(['socioeconomic_group', 'bmi_category']).size().reset_index(name='count')
/tmp/ipykernel_51936/1212948923.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_by_group = merged_bmi_df.groupby('socioeconomic_group').size()


It can be observed from the BMI Category Distribution by Socio-economic Group bar chart that the distribution of categories for lower and middle socio-economic groups is almost identical. They both have a skew to the more obese end of the bmi category sequence. The upper socio-economic group on the other hand have a more uniform distribution for all groups except the underweight bmi category.

This information might have both privacy and ethical concerns with regards to health insurance. It could be envisioned that insurance institutions might increase the cost of premiums for people who have higher BMI as they are at a higher risk of adverse health outcomes.

### Alcohol Use Analysis

Based on the estimated annual use of alcohol feature, we attempt to answer does one socio-economic group consume more alcohol than the any of the others.

In [16]:
# Merge demographic and alcohol use data
merged_alcohol_df = pd.merge(demo_df, alcohol_use_df, on='seqn', how='inner')
merged_alcohol_df.dropna(subset=["annual_use", SOCIOECONOMIC_GROUP], inplace=True)

# Calculate quartiles and IQR for annual_use
Q1 = merged_alcohol_df['annual_use'].quantile(0.25)
Q3 = merged_alcohol_df['annual_use'].quantile(0.75)
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR 
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
merged_alcohol_df = merged_alcohol_df[
    (merged_alcohol_df['annual_use'] >= lower_bound) & 
    (merged_alcohol_df['annual_use'] <= upper_bound)
]



groups = sorted(list(merged_alcohol_df[SOCIOECONOMIC_GROUP].unique()))

# compute quantiles
qs = merged_alcohol_df.groupby(SOCIOECONOMIC_GROUP)["annual_use"].quantile([0.25, 0.5, 0.75])
qs = qs.unstack().reset_index()
qs.columns = [SOCIOECONOMIC_GROUP, "q1", "q2", "q3"]

# compute IQR outlier bounds
iqr = qs.q3 - qs.q1
qs["upper"] = qs.q3 + 1.5*iqr
qs["lower"] = np.maximum(0, qs.q1 - 1.5*iqr) # cannot have lower less than zero
df = pd.merge(merged_alcohol_df, qs, on=SOCIOECONOMIC_GROUP, how="left")

source = ColumnDataSource(qs)

print(f"Groups: {groups}")


hover = HoverTool(tooltips=[
    ('Group', '@' + SOCIOECONOMIC_GROUP),
    ('Median', '@q2{0.0}'),
    ('Q1', '@q1{0.0}'),
    ('Q3', '@q3{0.0}'),
    ('Upper bound', '@upper{0.0}'),
    ('Lower bound', '@lower{0.0}')
])

p = figure(x_range=groups, 
           title="Annual Alcohol Use by Socio-economic Group",
           background_fill_color="#eaefef", 
           y_axis_label="Annual Use",
           tools=[hover, "pan", "box_zoom", "reset", "save"])

# outlier range
whisker = Whisker(base=SOCIOECONOMIC_GROUP, upper="upper", lower="lower", source=source)
whisker.upper_head.size = whisker.lower_head.size = 20
p.add_layout(whisker)

# quantile boxes
cmap = factor_cmap(SOCIOECONOMIC_GROUP, "TolRainbow7", groups)
p.vbar(SOCIOECONOMIC_GROUP, 0.7, "q2", "q3", source=source, color=cmap, line_color="black")
p.vbar(SOCIOECONOMIC_GROUP, 0.7, "q1", "q2", source=source, color=cmap, line_color="black")

# outliers with hover
outliers = df[~df.annual_use.between(df.lower, df.upper)]
outlier_source = ColumnDataSource(outliers)
outlier_hover = HoverTool(tooltips=[
    ('Group', '@' + SOCIOECONOMIC_GROUP),
    ('Annual Use', '@annual_use{0.0}')
], renderers=[p.scatter(SOCIOECONOMIC_GROUP, "annual_use", source=outlier_source, size=6, color="black", alpha=0.3)])
p.add_tools(outlier_hover)

p.xgrid.grid_line_color = None
p.axis.major_label_text_font_size="14px"
p.axis.axis_label_text_font_size="12px"

show(p)

/tmp/ipykernel_51936/3291843410.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  qs = merged_alcohol_df.groupby(SOCIOECONOMIC_GROUP)["annual_use"].quantile([0.25, 0.5, 0.75])


Groups: ['lower', 'middle', 'upper']


From the Annual Alcohol Use by Socio-economic Group box plots, it can be observed that the alcohol use between the lower and middle groups are almost identical. However the upper group is estimated to consume more alcohol annually than the other two groups.

Based on this finding, an insurance company might implement a policy to increase the risk premium for higher income individuals as they tend to consume more alcohol which in turn may impact their future health outcomes.

## Medical Condition Analysis

Using the medical condition dataset, we try to determine if there is a higher prevalence of heart conditions in one socio-economic group over another.

In [17]:

# Merge medical conditions with demo data to get socioeconomic groups
merged_medical_df = pd.merge(medical_conditions_df, demo_df[[SEQN, SOCIOECONOMIC_GROUP, "gender", "age"]], on=SEQN, how='left')

# Calculate percentage with heart condition by socioeconomic group
grouped = merged_medical_df.groupby(SOCIOECONOMIC_GROUP)['heart_condition'].agg(
    percentage=lambda x: (x == 1).mean() * 100,
    count=lambda x: len(x)  # Add count for hover tool
).reset_index()

source = ColumnDataSource(grouped)

hover = HoverTool(tooltips=[
    ('Socioeconomic Group', '@' + SOCIOECONOMIC_GROUP),
    ('Percentage', '@percentage{0.1f}%'),
    ('Sample Size', '@count')
])

p = figure(x_range=groups, height=400, title="Percentage with Heart Condition by Socio-economic Group",
           tools=[hover, "pan", "box_zoom", "reset", "save"], 
           background_fill_color="#eaefef",
           x_axis_label="Socioeconomic Group", 
           y_axis_label="Percentage with Heart Condition",
           y_range=(0, 25))  # Set y-axis range from 0 to 100

cmap = factor_cmap(SOCIOECONOMIC_GROUP, "TolRainbow7", groups)
p.vbar(x=SOCIOECONOMIC_GROUP, top='percentage', width=0.7, source=source,
       line_color="black", fill_color=cmap)


p.xgrid.grid_line_color = None
p.axis.major_label_text_font_size = "14px"
p.axis.axis_label_text_font_size = "12px"

show(p)

/tmp/ipykernel_51936/1585286728.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = merged_medical_df.groupby(SOCIOECONOMIC_GROUP)['heart_condition'].agg(


There is no significant differences in the percentage of people in each socio-economic group suffering from heart conditions. The upper group has a slight reduction however further statistical tests should be applied to determine if this is significant or not.

The information contained in this dataset has important privacy concerns that should be kept in mind. If the participants identities weren't kept secret sufficiently, or third parties were able to guess the identities of participants, it might have adverse impacts on their ability to obtain private health insurance coverage in the United States.

Next we will to determine if there is a difference in the prevalence of heart conditions between the two genders and if there is a difference between age groups.

In [18]:


# Group by gender and heart condition
sex_group = merged_medical_df.groupby(['gender', 'age', 'heart_condition'])

# Calculate percentages for each gender and age group
female_data = []
male_data = []

bin_width = 10
bins = np.arange(0, 100, bin_width)

for i in range(len(bins)-1):
    start_age = bins[i]
    end_age = bins[i+1]
    
    # Female percentages
    female_in_range = merged_medical_df[(merged_medical_df['gender'] == 'Female') & 
                                      (merged_medical_df['age'] >= start_age) & 
                                      (merged_medical_df['age'] < end_age)]
    female_total = len(female_in_range)
    if female_total > 0:
        female_pct = ((female_in_range['heart_condition'] == 1).sum() / female_total) * 100
    else:
        female_pct = 0
    female_data.append(female_pct)
    
    # Male percentages  
    male_in_range = merged_medical_df[(merged_medical_df['gender'] == 'Male') & 
                                    (merged_medical_df['age'] >= start_age) & 
                                    (merged_medical_df['age'] < end_age)]
    male_total = len(male_in_range)
    if male_total > 0:
        male_pct = (male_in_range['heart_condition'] == 1).sum() / male_total * 100
    else:
        male_pct = 0
    male_data.append(male_pct)

m_hist = np.array(male_data)
f_hist = np.array(female_data)

edges = bins[1:]  # Define edges here

p = figure(title="Heart Condition by Gender and Age", height=600, width=800,
           x_range=(-max(m_hist.max(), f_hist.max()), max(m_hist.max(), f_hist.max())), 
           x_axis_label="Percentage with Heart Condition")

p.hbar(right=-m_hist, y=edges, height=bin_width*0.8, color=colors[1], line_width=0)
p.hbar(right=f_hist, y=edges, height=bin_width*0.8, color=colors[0], line_width=0)

for i, (pct, age) in enumerate(zip(f_hist, edges)):
    p.text(x=pct, y=edges[i], text=[f"{age-bin_width}-{age}yrs"],
           x_offset=5, y_offset=7, text_font_size="12px", text_color=DarkText[5])

p.xaxis.ticker = (-50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50)
p.xaxis.major_tick_out = 0
p.y_range.start = 3
p.ygrid.grid_line_color = None
p.yaxis.visible = False

p.xaxis.formatter = CustomJSTickFormatter(code="return Math.abs(tick);")

p.add_layout(Label(x=-15, y=94, text="Men", text_color=colors[1], x_offset=5))
p.add_layout(Label(x=2, y=94, text="Women", text_color=colors[0], x_offset=5))

show(p)



It can be seen from the Heart Condition by Gender and Age pyramid graphic, that there is indeed age and gender differences for the prevalence of heart conditions. There is a higher percentage of male respondents from the age of 50 and above who have heart conditions when compare to their female counterparts. The pyramid is also inverted, meaning that as one ages, they have a higher chance of acquiring a heart condition.

The health conditions data also provides information on cancer rates. We group this by socio-economic group to gleam whether there is a difference between these populations.

In [19]:
# Calculate percentage of cancer by socioeconomic group
merged_medical_df.dropna(subset=[SOCIOECONOMIC_GROUP], inplace=True)
cancer_by_group = merged_medical_df.groupby(SOCIOECONOMIC_GROUP).agg(
    cancer_count=('cancer', lambda x: (x == 1).sum()),
    total_count=('cancer', 'count')
).reset_index()

cancer_by_group['percentage'] = (cancer_by_group['cancer_count'] / cancer_by_group['total_count']) * 100
groups = sorted(list(merged_medical_df[SOCIOECONOMIC_GROUP].unique()))


p = figure(x_range=cancer_by_group['socioeconomic_group'].tolist(),
           height=400, width=600,
           title="Cancer Prevalence by Socioeconomic Group",
           x_axis_label='Socioeconomic Group',
           y_axis_label='Percentage with Cancer (%)',
           y_range=(0, 25))

cmap = factor_cmap(SOCIOECONOMIC_GROUP, "TolRainbow7", groups)
p.vbar(x='socioeconomic_group', 
       top='percentage',
       width=0.8,
       source=cancer_by_group,
       fill_color=cmap,
       line_color='black')

p.xgrid.grid_line_color = None
p.y_range.start = 0

hover = HoverTool(tooltips=[
    ('Group', '@socioeconomic_group'),
    ('Percentage', '@percentage{0.1f}%')
])
p.add_tools(hover)
p.xaxis.major_label_orientation = 0.7
show(p)


/tmp/ipykernel_51936/3339463692.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cancer_by_group = merged_medical_df.groupby(SOCIOECONOMIC_GROUP).agg(


From this we can surmise that middle and upper socio-economic groups have identical percentages of their population with cancer. However, the lower group has a slight decrease in cancer rate when compared to the other two groups. (12.3% versus 16.2%)

## Smoking - Cigarette Use Analysis

Here we try to answer the question, is there any difference in the use of cigarettes between the differing socio-economic groups?

In [20]:
merged_smoke_df = pd.merge(smoking_df,demo_df[[SEQN, SOCIOECONOMIC_GROUP, "gender", "age"]], on=SEQN, how='inner')
merged_smoke_df.dropna(subset=[SOCIOECONOMIC_GROUP], inplace=True)

# Calculate percentage of smokers by socioeconomic group
smoking_by_group = (
    merged_smoke_df.groupby('socioeconomic_group')
    .agg(
        total_count=('seqn', 'count'),
        smokers_count=('smoked_in_life', lambda x: (x == 1).sum())
    )
    .reset_index()
)
smoking_by_group['smoking_percentage'] = (
    smoking_by_group['smokers_count'] / smoking_by_group['total_count'] * 100
)

source = ColumnDataSource(smoking_by_group)

p = figure(
    x_range=smoking_by_group['socioeconomic_group'].tolist(),
    height=400,
    title='Percentage of People Who Have Smoked by Socio-economic Group',
    toolbar_location='right'
)

bars = p.vbar(
    x='socioeconomic_group',
    top='smoking_percentage',
    width=0.7,
    source=source,
    fill_color='#1f77b4',
    line_color='black'
)

p.xgrid.grid_line_color = None
p.xaxis.axis_label = 'Socio-economic Group'
p.yaxis.axis_label = 'Percentage of Smokers (%)'
p.xaxis.major_label_orientation = 0.7

hover = HoverTool(
    renderers=[bars],
    tooltips=[
        ('Group', '@socioeconomic_group'),
        ('Smoking Percentage', '@smoking_percentage{0.1f}%'),
        ('Number of Smokers', '@smokers_count'),
        ('Total Population', '@total_count')
    ]
)
p.add_tools(hover)

output_notebook()
show(p)

/tmp/ipykernel_51936/2330604987.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_smoke_df.groupby('socioeconomic_group')


Loading BokehJS ...

The Percentage of People Who Have Smoked by Socio-economic Group is the percentage of people in each socio-economic group who have indicated that they have smoked at least 100 cigarettes in their life time. From this, we can that in the upper group, only 33% of respondents have ever smoked, whilst in the lower and middle groups, 47% and 44% respectively have smoked. People in the upper socio-economic group have more of a tendency to be non-smokers than people in the lower and middle groups.

It would be useful to determine, for people who smoke, is there any difference between socio-economic groups in the annual amount smoked by individuals.

In [21]:


do_smoke_df = merged_smoke_df[merged_smoke_df["smoke_use"] > 0]

# Calculate Q1, Q3 and IQR for smoke_use
Q1 = do_smoke_df["smoke_use"].quantile(0.25)
Q3 = do_smoke_df["smoke_use"].quantile(0.75)
IQR = Q3 - Q1

# Filter out rows with smoke_use outside 1.5*IQR bounds
do_smoke_df = do_smoke_df[
    (do_smoke_df["smoke_use"] >= (Q1 - 1.5 * IQR)) & 
    (do_smoke_df["smoke_use"] <= (Q3 + 1.5 * IQR))
]

groups = sorted(list(do_smoke_df[SOCIOECONOMIC_GROUP].unique()))

# compute quantiles
qs = do_smoke_df.groupby(SOCIOECONOMIC_GROUP)["smoke_use"].quantile([0.25, 0.5, 0.75])
qs = qs.unstack().reset_index()
qs.columns = [SOCIOECONOMIC_GROUP, "q1", "q2", "q3"]

# compute IQR outlier bounds
iqr = qs.q3 - qs.q1
qs["upper"] = qs.q3 + 1.5*iqr
qs["lower"] = np.maximum(0, qs.q1 - 1.5*iqr)
df = pd.merge(do_smoke_df, qs, on=SOCIOECONOMIC_GROUP, how="left")

source = ColumnDataSource(qs)

hover = HoverTool(tooltips=[
    ('Group', '@' + SOCIOECONOMIC_GROUP),
    ('Median', '@q2{0.0}'),
    ('Q1', '@q1{0.0}'),
    ('Q3', '@q3{0.0}'),
    ('Upper bound', '@upper{0.0}'),
    ('Lower bound', '@lower{0.0}')
])

p = figure(x_range=groups, 
           title="Annual Cigarette Use by Socio-economic Group",
           background_fill_color="#eaefef", 
           y_axis_label="Smoke Use",
           tools=[hover, "pan", "box_zoom", "reset", "save"],
           width=800,
           height=600)

whisker = Whisker(base=SOCIOECONOMIC_GROUP, upper="upper", lower="lower", source=source)
whisker.upper_head.size = whisker.lower_head.size = 20
p.add_layout(whisker)


cmap = factor_cmap(SOCIOECONOMIC_GROUP, "TolRainbow7", groups)
p.vbar(SOCIOECONOMIC_GROUP, 0.7, "q2", "q3", source=source, color=cmap, line_color="black")
p.vbar(SOCIOECONOMIC_GROUP, 0.7, "q1", "q2", source=source, color=cmap, line_color="black")


outliers = df[~df.smoke_use.between(df.lower, df.upper)]
outlier_source = ColumnDataSource(outliers)
outlier_hover = HoverTool(tooltips=[
    ('Group', '@' + SOCIOECONOMIC_GROUP),
    ('Smoke Use', '@smoke_use{0.0}')
], renderers=[p.scatter(SOCIOECONOMIC_GROUP, "smoke_use", source=outlier_source, size=6, color="black", alpha=0.3)])
p.add_tools(outlier_hover)

p.xgrid.grid_line_color = None
p.axis.major_label_text_font_size="14px"
p.axis.axis_label_text_font_size="12px"

p.y_range.start = -100
p.y_range.end = max(qs.upper) + 100

show(p)

/tmp/ipykernel_51936/3245923364.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  qs = do_smoke_df.groupby(SOCIOECONOMIC_GROUP)["smoke_use"].quantile([0.25, 0.5, 0.75])


The Annual Cigarette Use by Socio-economic Group box plots demonstrates that the is no real difference in the average annual amount people smoke in the different socio-economic categories. The dispersion of annual cigarette use is also almost identical between the middle and upper groups. However, there is a difference in the dispersion for the lower group in that individuals who smoke more than average tend to smoke more annually than their counterparts in the middle and upper groups.

# Conclusion

The NHANES datasets have provided valuable insights into the health metrics and life style choices for people within distinct socio-economic groups. According to the data it can be seen that socio-economic status appears to influence certain behaviours and outcomes, such as alcohol and smoking use, whereas its has little impact on other health indicators such as BMI.

Key findings include:
- BMI Distribution - Socio-economic groups exhibit similar BMI distributions with the exception of the upper group having a slightly more uniform distribution when compared to the other groups
- Alcohol Use - Annual alcohol consumption was shown to be higher in the upper socio-economic group when compared to the lower and middle groups.
- Medical Conditions - There was no significant difference in heart conditions between the socio-economic groups. There was a slight decrease in the prevalence of cancer in the lower socio-economic group when compared to the other two groups
- Smoking Use - A higher percentage of individuals in the lower socio-economic group reported to have smoked at least 100 cigarettes in their lifetime when compared to other groups. The upper group reported 13% less than the lower group in having smoked at least 100 cigarettes. Amongst smokers, the annual consumption levels were comparable across the groups, with a slight dispersion increase in the lower group.

Ethical and privacy concerns surrounding health data such as the NHANES dataset must always be highlighted as the potential misuse of such data could have detrimental impacts on people's lives. Proper safe guards must be maintained to ensure and protect the identities of participants and to ensure that nefarious uses of such data cannot occur.

# Reference
1. Centers for Disease Control and Prevention, National Center for Health Statistics. (2020). National Health and Nutrition Examination Survey: 2021–2023 Data Documentation, Codebook, and Frequencies. Retrieved from https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?Cycle=2021-2023
2. Bokeh Development Team (2018). Bokeh: Python library for interactive visualization URL http://www.bokeh.pydata.org.
3. Wikipedia contributors. (2024, December 12). Body mass index. In Wikipedia, The Free Encyclopedia. Retrieved 02:24, January 3, 2025, from https://en.wikipedia.org/w/index.php?title=Body_mass_index&oldid=1262690767


# Appendix 1 - Demographics Dataset Features

* SEQN - Respondent sequence number
* SDDSRVYR - Data release cycle
* RIDSTATR - Interview/Examination status
* RIAGENDR - Gender
* RIDAGEYR - Age in years at screening
* RIDAGEMN - Age in months at screening - 0 to 24 mos
* RIDRETH1 - Race/Hispanic origin
* RIDRETH3 - Race/Hispanic origin w/ NH Asian
* RIDEXMON - Six-month time period
* RIDEXAGM - Age in months at exam - 0 to 19 years
* DMQMILIZ - Served active duty in US Armed Forces
* DMDBORN4 - Country of birth
* DMDYRUSR - Length of time in US
* DMDEDUC2 - Education level - Adults 20+
* DMDMARTZ - Marital status
* RIDEXPRG - Pregnancy status at exam
* DMDHHSIZ - Total number of people in the Household
* DMDHRGND - HH ref person’s gender
* DMDHRAGZ - HH ref person’s age in years
* DMDHREDZ - HH ref person’s education level
* DMDHRMAZ - HH ref person’s marital status
* DMDHSEDZ - HH ref person’s spouse’s education level
* WTINT2YR - Full sample 2-year interview weight
* WTMEC2YR - Full sample 2-year MEC exam weight
* SDMVSTRA - Masked variance pseudo-stratum
* SDMVPSU - Masked variance pseudo-PSU
* INDFMPIR - Ratio of family income to poverty

# Appendix 2 - Body Measure Dataset Features

* SEQN - Respondent sequence number
* BMDSTATS - Body Measures Component Status Code
* BMXWT - Weight (kg)
* BMIWT - Weight Comment
* BMXRECUM - Recumbent Length (cm)
* BMIRECUM - Recumbent Length Comment
* BMXHEAD - Head Circumference (cm)
* BMIHEAD - Head Circumference Comment
* BMXHT - Standing Height (cm)
* BMIHT - Standing Height Comment
* BMXBMI - Body Mass Index (kg/m**2)
* BMDBMIC - BMI Category - Children/Youth
* BMXLEG - Upper Leg Length (cm)
* BMILEG - Upper Leg Length Comment
* BMXARML - Upper Arm Length (cm)
* BMIARML - Upper Arm Length Comment
* BMXARMC - Arm Circumference (cm)
* BMIARMC - Arm Circumference Comment
* BMXWAIST - Waist Circumference (cm)
* BMIWAIST - Waist Circumference Comment
* BMXHIP - Hip Circumference (cm)
* BMIHIP - Hip Circumference Comment

# Appendix 3 - Alcohol Use Dataset Features

* SEQN - Respondent sequence number
* ALQ111 - Ever had a drink of any kind of alcohol
* ALQ121 - Past 12 mos how often drink alc bev
* ALQ130 - Avg # alcoholic drinks/day/past 12 mos
* ALQ142 - # days have 4/5 drinks/past 12 mos
* ALQ270 - # times 4/5 drinks in 2hrs/past 12 mos
* ALQ280 - # times 8+ drinks in 1 day/past 12 mos
* ALQ151 - Ever have 4/5 or more drinks every day
* ALQ170 - # times 4/5 drinks on occasion/past mo

# Appendix 4 - Medical Conditions Dataset Features

* SEQN - Respondent sequence number
* MCQ010 - Ever been told you have asthma
* MCQ035 - Still have asthma
* MCQ040 - Had asthma attack in past year
* MCQ050 - Emergency care visit for asthma/past yr
* AGQ030 - Did SP have episode of hay fever/past yr
* MCQ053 - Taking treatment for anemia/past 3 mos
* MCQ145 - CHECK ITEM
* MCQ149 - Menstrual periods started yet?
* MCQ157 - CHECK ITEM
* MCQ160a - Doctor ever said you had arthritis
* MCQ195 - Which type of arthritis was it?
* MCQ160b - Ever told had congestive heart failure
* MCQ160c - Ever told you had coronary heart disease
* MCQ160d - Ever told you had angina/angina pectoris
* MCQ160e - Ever told you had heart attack
* MCQ160f - Ever told you had a stroke
* MCQ160m - Ever told you had thyroid problem
* MCQ170m - Do you still have thyroid problem
* MCQ160p - Ever told you had COPD, emphysema, ChB
* MCQ160l - Ever told you had any liver condition
* MCQ170l - Do you still have a liver condition
* MCQ500 - Ever told you had any liver condition
* MCQ510a - Liver condition: Fatty liver
* MCQ510b - Liver condition: Liver fibrosis
* MCQ510c - Liver condition: Liver cirrhosis
* MCQ510d - Liver condition: Viral hepatitis
* MCQ510e - Liver condition: Autoimmune hepatitis
* MCQ510f - Liver condition: Other liver disease
* MCQ515 - CHECK ITEM
* MCQ550 - Has DR ever said you have gallstones
* MCQ560 - Ever had gallbladder surgery?
* MCQ220 - Ever told you had cancer or malignancy
* MCQ230a - 1st cancer - what kind was it?
* MCQ230b - 2nd cancer - what kind was it?
* MCQ230c - 3rd cancer - what kind was it?
* MCQ230d - More than 3 kinds of cancer
* OSQ230 - Any metal objects inside your body?

# Appendix 5 - Smoking - Cigarette Use Dataset Features

* SEQN - Respondent sequence number
* SMQ020 - Smoked at least 100 cigarettes in life
* SMQ040 - Do you now smoke cigarettes?
* SMD641 - # days smoked cigs during past 30 days
* SMD650 - Avg # cigarettes/day during past 30 days
* SMD100MN - Cigarette Menthol indicator
* SMQ621 - Cigarettes smoked in entire life
* SMD630 - Age first smoked whole cigarette
* SMAQUEX2 - Questionnaire Mode Flag